In [4]:
!pip install scikit-learn==1.3.0



  ERROR: Could not find a version that satisfies the requirement scikit-learn==1.3.0 (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.22, 0.22.1, 0.22.2, 0.22.2.post1, 0.23.0, 0.23.1, 0.23.2, 0.24.0, 0.24.1, 0.24.2, 1.0, 1.0.1, 1.0.2)
ERROR: No matching distribution found for scikit-learn==1.3.0


In [1]:
import sklearn

print(sklearn.__version__)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:

!pip install tk


In [27]:
import joblib
import pandas as pd
import numpy as np

# Load the trained models
voting_ensemble = joblib.load('E:/NITW/Project/MODELS/ML/Voting_Ensemble_CICIOT2023.pkl')
random_forest = joblib.load('E:/NITW/Project/MODELS/ML/Random_forest_CICIOT2023.pkl')
adaboost = joblib.load('E:/NITW/Project/MODELS/ML/adaboost_CICIOT2023.pkl')
bagging = joblib.load('E:/NITW/Project/MODELS/ML/bagging_CICIOT2023.pkl')


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

def get_csv_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    return file_path

# Ask the user to select a CSV file
csv_file_path = get_csv_file()

if csv_file_path:
    # Load the test data
    data = pd.read_csv(csv_file_path)

    # Call the check_dataset function with 'data' and 'voting_ensemble'
    result = check_dataset(data, voting_ensemble)
    if result:
        # Proceed with prediction using the pre-trained model
        predictions = voting_ensemble.predict(data)
        # Further processing and evaluation can be done here
    else:
        print('Dataset does not meet the necessary conditions for prediction.')
else:
    print('No CSV file selected. Exiting...')


In [34]:
#user uploads the data.csv file from front-end
# Load the test data
data = pd.read_csv('E:/NITW/Project/Datasets/CICIOT2023_EXTRACTED/92.csv')


In [35]:
print("Number of rows in the DataFrame:", len(data))

Number of rows in the DataFrame: 211834


In [40]:
import pandas as pd

def check_dataset(data,for_model ,target_class=None ):
    if target_class is None:
        target_class = data.columns[-1]
        print(f"Considering the last column '{target_class}' as the target.")
    X = data.drop(columns=[target_class])
    y = data[target_class]
    unique_classes = y.unique()

    # Check the X feature vector of the dataset
    if not set(X.columns) == set(for_model.feature_names_in_):
        print('Input feature vector mismatch:')
        actual_columns_set = set(X.columns)
        trained_columns_set = set(for_model.feature_names_in_)
        mismatched_columns = actual_columns_set.symmetric_difference(trained_columns_set)
        for column in mismatched_columns:
            print(f"Column '{column}' is present in {'X' if column in actual_columns_set else 'for_model.feature_names_in_'} but not in the other.")
        return False

    # Check if the target classes match the model's classes
    if not set(unique_classes) == set(for_model.classes_):
        print('Target classes mismatch:')
        actual_classes_set = set(unique_classes)
        trained_classes_set = set(for_model.classes_)
        mismatched_classes = actual_classes_set.symmetric_difference(trained_classes_set)
        for target_class in mismatched_classes:
            print(f"Target class '{target_class}' is present in {'data' if target_class in actual_classes_set else 'for_model.classes_'} but not in the other.")
        return False

    # Check for null values in the dataset
    if data.isnull().any().any():
        print('Null values found in the dataset.')
        return False

#     # Check for data type mismatches
#     if not data.dtypes.equals(for_model.dtypes_in_):
#         print('Data type mismatches in the dataset.')
#         return False

    # Add any other necessary checks here, if required

    return True




In [41]:
#checking for only one model since all are trained on same data
result = check_dataset(data, voting_ensemble)
if result:
    print("Proceed with prediction using the pre-trained model")
       # Further processing and evaluation can be done here
else:
    print('Dataset does not meet the necessary conditions for prediction.')

# Display the number of rows in the DataFrame

Considering the last column 'label' as the target.
Proceed with prediction using the pre-trained model


In [43]:
while True:
    try:
        # Ask the user to pick a row
        selected_row = int(input("Pick a single packet from the passed dataset  (Enter the row number between 0 and {}): ".format(len(data)-1)))
        if selected_row < 0 or selected_row >= len(data):
            raise ValueError
    except ValueError:
        print("Invalid input. Please enter a valid row number.")
        continue

    # Extract the selected row data
    sampledata = data.iloc[selected_row]
    sampleInput = sampledata.drop('label')
    sampleTrue = sampledata['label']

    # Get the feature names from the voting_ensemble model
    feature_names = voting_ensemble.feature_names_in_

    # Convert sampleInput to a DataFrame and set feature names
    sampleInput = pd.DataFrame([sampleInput], columns=feature_names)

    # Perform predictions for each model
    y_pred_voting = voting_ensemble.predict(sampleInput)
    y_pred_rf = random_forest.predict(sampleInput)
    y_pred_ab = adaboost.predict(sampleInput)
    y_pred_bag = bagging.predict(sampleInput)

    # Convert the predicted label (y_pred) to a single scalar value
    pred_voting = y_pred_voting[0]
    pred_rf = y_pred_rf[0]
    pred_ab = y_pred_ab[0]
    pred_bag = y_pred_bag[0]

    # Print the predicted results
    print("\nPredicted Results:")
    print("Voting Ensemble Prediction:", pred_voting)
    print("Random Forest Prediction:", pred_rf)
    print("AdaBoost Prediction:", pred_ab)
    print("Bagging Prediction:", pred_bag)

    # Print the true label
    print("True Label:", sampleTrue)

    # Calculate and print accuracies
    ac_voting = np.mean(pred_voting == sampleTrue)
    ac_rf = np.mean(pred_rf == sampleTrue)
    ac_ab = np.mean(pred_ab == sampleTrue)
    ac_bag = np.mean(pred_bag == sampleTrue)

    print("\nAccuracies:")
    print("Voting Ensemble Accuracy:", ac_voting)
    print("Random Forest Accuracy:", ac_rf)
    print("AdaBoost Accuracy:", ac_ab)
    print("Bagging Accuracy:", ac_bag)

    # Ask the user if they want to continue making predictions
    response = input("Do you want to make another prediction? (y/n): ")
    if response.lower() != 'y':
        break

Pick a single packet from the passed dataset  (Enter the row number between 0 and 211833): 211882
Invalid input. Please enter a valid row number.
Pick a single packet from the passed dataset  (Enter the row number between 0 and 211833): 211832

Predicted Results:
Voting Ensemble Prediction: DDoS-UDP_Flood
Random Forest Prediction: DDoS-UDP_Flood
AdaBoost Prediction: DDoS-UDP_Flood
Bagging Prediction: DDoS-UDP_Flood
True Label: DDoS-UDP_Flood

Accuracies:
Voting Ensemble Accuracy: 1.0
Random Forest Accuracy: 1.0
AdaBoost Accuracy: 1.0
Bagging Accuracy: 1.0
Do you want to make another prediction? (y/n): n
